# *Balances de energía*
## Resolución de un balance de energía empleando vapor 

Se utiliza vapor de agua para calentar un medio  en un proceso en continuo.  Un vapor saturado con un caudal mV a una temperatura T1 entra en un serpentín en el tanque donde se encuentra el medio (caudal másico mL) y condensa completamente.  El medio líquido entra al tanque a una temperatura T2 y lo abandona a T3.  Las pérdidas de calor de la camisa al exterior se estiman en Qp. 

### El diagrama del proceso es el siguiente:


 <img src="./Imagenes/Sistema_Balance_Energia.jpg" height="400" width="400"/>

### Nomenclatura:

$m_V$: Caudal másico de vapor de calefacción (kg/h)<br>
$m_L$: Caudal másico del medio (kg/h)<br>
$h_1$: Entalpía del líquido saturado (kJ/kg))<br>
$h_2$: Entalpía del vapor saturado (kJ/kg)<br>
$h_3$: Entalpía del medio caliente (kJ/kg))<br>
$h_4$: Entalpía del medio frío (kJ/kg)<br>
$\lambda$: Calor latente de vaporización del agua (kJ/kg)<br>
$C_p$: Calor específico del medio (kJ/kg·K)<br>
$T_1$: Temperatura del vapor saturado (ºC)<br>
$T_3$: Temperatura del medio caliente (ºC)<br>
$T_4$: Temperatura del medio frío (ºC)<br>
$Q_p$: Pérdidas de calor (kJ/h)<br>









In [ ]:
%matplotlib inline
import numpy as np
from scipy import optimize
from scipy.integrate import ode
from scipy.interpolate import interp1d
import matplotlib.pyplot as mpl
import matplotlib.patches as patches
import random
import math
import pandas as pd
import IPython.core.display as di
import notebook
from IPython.display import clear_output, display, HTML, Image,Math, Latex
from IPython.external import mathjax

In [ ]:
def P_Saturacion(T):
    #Temperatura en ºC y P en bar
    T_Kelvin=T + 273.15
    if T_Kelvin>=255.9 and T_Kelvin<=373.0:
        A=4.6543
        B=1435.264
        C=-64.848
        P = 10 ** (A - (B / (T_Kelvin + C)))

    elif  T_Kelvin>=379.0 and  T_Kelvin<=573.0:
        A= 3.55959
        B=643.748
        C=-198.043
        P = 10 ** (A - (B / (T_Kelvin + C)))

    elif T_Kelvin>373.0 and  T_Kelvin<379.0:
        x1=np.linspace(89.85, 99.85, num=21)
        x2=np.linspace(105.85, 115.85, num=21)
        x = np.append(x1, x2)
        vfunc = np.vectorize(P_Saturacion)
        y=vfunc(x)
        x=x+273.15
        f = interp1d(x, y, kind='quadratic')
        P=float(f(T_Kelvin))

    else:
        raise ValueError('Temperatura fuera del intervalo válido')


    return P

In [1]:
def T_Saturacion(P):
    #P en kPa
    T_sat=(42.6776-(3892.7/(np.log(P/1000)-9.48654)))-273.15
    return T_sat


def Entalpia_AguaLiquida(T):
    #Correlación válida de 0.01 a 200ºC
    H_agua_Saturada=-0.033635409+4.207557011*T-6.200339E-4*T**2+4.459374E-6*T**3 #Entalpia kJ/kg
    return H_agua_Saturada

def Entalpia_Vapor(T):
    # Correlación válida de 0.01 a 200ºC
    H_vapor_Saturado=2501.6898445+1.806916015*T+5.087717E-4*T**2-1.1221E-5*T**3
    return H_vapor_Saturado



def Calor_Latente_Vaporizacion(T):
    # Correlación válida de 0.01 a 200ºC
    Lamda=2501.897149-2.407064037*T+1.192217E-3*T**2-1.5863E-5*T**3 #kJ/kg
    return Lamda

def Cp_AguaLiquida(T):
    T_Kelvin=(T+273.15)/1000
    A = -203.6060
    B = 1523.290
    C = -3196.413
    D = 2474.455
    E = 3.855326
    Cp = A + B * T_Kelvin + C * T_Kelvin ** 2. + D * T_Kelvin ** 3. + E / (T_Kelvin ** 2.)  # En J/mol·K
    Cp_Masa = Cp / 18  # En kJ/kg·K
    return Cp_Masa


def Cp_AguaVapor(T):
    T_Kelvin=(T+273.15)/1000
    A = -203.6060
    B = 1523.290
    C = -3196.413
    D = 2474.455
    E = 3.855326
    Cp = A + B * T_Kelvin + C * T_Kelvin ** 2. + D * T_Kelvin ** 3. + E / (T_Kelvin ** 2.)  # En J/mol·K
    Cp_Masa = Cp / 18  # En kJ/kg·K
    return Cp_Masa


def Cp_vapor(T,P):
    # T en ºC y P en Pa
    # Correlación válida de 0  a 200 ºC y 200 kPa

    if T<50:
        AE = 1877.2
        BE = -0.49545
        CE = 8.1818E-3
        AF = 22.537
        BF= 0.49321
        CF=0.048927
    else:
        AE = 1856.1
        BE = 0.28056
        CE = 6.9444E-4
        AF = 22.537
        BF= 0.49321
        CF=0.048927
    Ptr=611.657
    Cp_Masa =(AE+BE*T +CE*T**2+(1/(AF+BF*T +CF*T**2))*(P-Ptr))/1000
     # En kJ/kg·K
    return Cp_Masa

# mV es el caudal de vapor (kg/h)
# mL es el caudal del medio (propiedades agua líquida) (kg/h)
# Tv es la temperatura del vapor saturado (ºC)
# Pv es la presión del vapor saturado (kPa)
# Qp es el caudal de pérdidas de calor (kW)
# T1 es la temperatura de entrada del medio (ºC)
# T2 es la temperatura de salida del medio (ºC)
# lamdaV es el calor latente de vaporización del agua (kJ/kg)
# CpV es la capacidad calorífica del vapor  (kJ/kg ºC)
# CpL es la capacidad calorífica del agua líquida (kJ/kg ºC)



def generador_parametros():  # generación de parámetros del problema

    mV=random.randint(100, 300.)
    Tv =round(random.uniform(100, 150),1)
    Pv=round(P_Saturacion(Tv),3)
    lamdaV=round(Calor_Latente_Vaporizacion(Tv))
    h2=round(Entalpia_Vapor(Tv))
    h1= round(Entalpia_AguaLiquida(Tv))
    Qcedido=round((mV*lamdaV))
    Qp=round((round(random.uniform(0.05, 0.2),3)*Qcedido))
    Qganado=round(Qcedido-Qp)
    T4 = round(random.uniform(40, 100), 1)
    incrT=round(random.uniform(10, 40), 1)
    T3=round(T4-incrT,1)
    Tmedia=incrT/2
    CpL=round(Cp_AguaLiquida(Tmedia),2)
    h3=round(Entalpia_AguaLiquida(T3))
    h4=round(Entalpia_AguaLiquida(T4))
    mL=round(Qganado/(CpL*incrT))
    
    return mV,Tv,Pv,lamdaV,h2,h1,Qcedido,Qp,Qganado,T4,incrT,T3,CpL,h3,h4,mL



#Definir casos: generar variables y sus valores para mostrarlos

mV,Tv,Pv,lamdaV,h2,h1,Qcedido,Qp,Qganado,T4,incrT,T3,CpL,h3,h4,mL=generador_parametros()

caso=random.randint(1,2)

if caso==1: #determinar un caudal (vapor o medio) conocidas las entalpías del vapor y del medio
    subcaso=random.randint(1,2)
    
    if subcaso==1:
        display(HTML('<h3 style="color:#000000"><strong>Conocido el caudal másico de vapor empleado y los \
        datos mostrados en la siguiente tabla, determinar el caudal de calor ganado por el medio y su caudal \
        másico.</strong></h3>'))
        
        variablesConocidasNombres=[r'$m_V$ (kg/h)',r'$T_1$ (ºC)',r'$h_1$ (kJ/kg) ', \
        r'$h_2$ (kJ/kg) ',r'$h_4$ (kJ/kg)' , r'$h_3$ (kJ/kg)' ,\
        r'$Q_p$  (kJ/h)']        
        ValoresMostrados=[mV, Tv, h1, h2, h3, h4, Qp]
        
        data = dict(zip(variablesConocidasNombres, ValoresMostrados))
        values = pd.DataFrame(data, index=['Datos'],columns=variablesConocidasNombres)
        display(values)
        
    if subcaso==2:
        display(HTML('<h3 style="color:#000000"><strong>Conocido el caudal másico del medio calentado y los \
        datos mostrados en la siguiente tabla, determinar el caudal de calor cedido por el vapor y su caudal \
        másico.</strong></h3>'))
        
        variablesConocidasNombres=[r'$m_L$ (kg/h)',r'$h_1$ (kJ/kg) ', \
        r'$h_2$ (kJ/kg) ',r'$h_4$ (kJ/kg)' , r'$h_3$ (kJ/kg)' ,\
        r'$Q_p$  (kJ/h)']        
        ValoresMostrados=[mL, Tv, h1, h2, h3, h4, Qp]
        
        data = dict(zip(variablesConocidasNombres, ValoresMostrados))
        values = pd.DataFrame(data, index=['Datos'],columns=variablesConocidasNombres)
        display(values)



if caso==2: #determinar un caudal (vapor o medio) conocido el calor latente de vapor y el específico del medio
    subcaso=random.randint(1,2)
    
    if subcaso==1:
        display(HTML('<h3 style="color:#000000"><strong>Conocido el caudal másico de vapor empleado y los \
        datos mostrados en la siguiente tabla, determinar el caudal de calor ganado por el medio y su caudal \
        másico.</strong></h3>'))
        
        variablesConocidasNombres=[r'$m_V$ (kg/h)',r'$T_1$ (ºC)',r'$\lambda$ (kJ/kg) ',\
        r'$C_p$ (kJ/kg·K) ',r'$T_4$ (ºC)', r'$T_3$ (ºC)', r'$Q_p$(kJ/h)']        
        ValoresMostrados=[mV, Tv, lamdaV,CpL ,T3 , T4, Qp]
        
        data = dict(zip(variablesConocidasNombres, ValoresMostrados))
        values = pd.DataFrame(data, index=['Datos'],columns=variablesConocidasNombres)
        display(values)
        
    if subcaso==2:
        display(HTML('<h3 style="color:#000000"><strong>Conocido el caudal másico del medio calentado y los \
        datos mostrados en la siguiente tabla, determinar el caudal de calor ganado por el medio y su caudal \
        másico.</strong></h3>'))
        
        
        variablesConocidasNombres=[r'$m_L$ (kg/h)',r'$T_1$ (ºC)',r'$\lambda$ (kJ/kg) ',\
        r'$C_p$ (kJ/kg·K) ',r'$T_4$ (ºC)', r'$T_3$ (ºC)', r'$Q_p$  (kJ/h)']        
        ValoresMostrados=[mL, Tv, lamdaV,CpL ,T3 , T4, Qp]
        
        data = dict(zip(variablesConocidasNombres, ValoresMostrados))
        values = pd.DataFrame(data, index=['Datos'],columns=variablesConocidasNombres)
        display(values)

    
display(HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Mostrar código')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Esconder código')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Mostrar código"></form>'''))

,$m_L$ (kg/h),$h_1$ (kJ/kg),$h_2$ (kJ/kg),$h_4$ (kJ/kg),$h_3$ (kJ/kg),$Q_p$ (kJ/h)
Datos,10192,146.2,616,2742,275,324


In [2]:
display(HTML('<h3 style="color:#000000"><strong>Los resultados del balance de energía son:</strong></h3>'))

    
if subcaso==1:        
        
    variablesConocidasNombres=[r'$Q_{ganado}$  (kJ/h)',r'$m_L$ (kg/h)', r'$Q_{cedido}$ (kJ/h)']        
    ValoresMostrados=[Qganado, mL, Qcedido]
        
    data1 = dict(zip(variablesConocidasNombres, ValoresMostrados))
    values1 = pd.DataFrame(data1, index=['Resultados'],columns=variablesConocidasNombres)
    display(values1)
        
if subcaso==2:                   
       
    variablesConocidasNombres=[r'$Q_{cedido}$  (kJ/h)',r'$m_V$ (kg/h)', r'$Q_{ganado}$ (kJ/h)']        
    ValoresMostrados=[Qcedido, mV, Qganado]

    data1 = dict(zip(variablesConocidasNombres, ValoresMostrados))
    values1 = pd.DataFrame(data1, index=['Resultados'],columns=variablesConocidasNombres)
    display(values1)


,$Q_{cedido}$ (kJ/h),$m_V$ (kg/h),$Q_{ganado}$ (kJ/h)
Resultados,542130,255,502012


In [3]:
display(HTML('''

<footer id="attribution" style="float:right; color:#999; background:#fff;">
Programado con Jupyter Notebook en Python 3.6. </footer>'''))